### 1. PCA
Метод главных компонент направлен на снижение размерности данных путем нахождения такого направления в многомерном пространстве, вдоль которого дисперсия проекций объектов достигает максимума. Формально, для заданного набора точек в $ \mathbb{R}^n $ ищется прямая, максимизирующая разброс данных при проектировании на нее. Этот разброс количественно выражается через дисперсию.


### 2. Роль ковариационной матрицы
Ковариационная матрица $ \Sigma $ описывает структуру вариации данных во всех возможных направлениях. Для произвольного единичного вектора $ \mathbf{w} $ (т.е. $ \|\mathbf{w}\| = 1 $), задающего направление, дисперсия проекций точек на это направление определяется выражением:
$$
D = \mathbf{w}^{\top} \Sigma \mathbf{w}.
$$
Таким образом, задача PCA сводится к поиску $ \mathbf{w} $, максимизирующего данную квадратичную форму при указанном ограничении на норму.


### 3. Собственные векторы и собственные значения
Ковариационная матрица $ \Sigma $ является симметричной и положительно полуопределенной, что гарантирует существование ортонормированного базиса из собственных векторов. Для каждого собственного вектора $ \mathbf{v}_k $ выполняется:
$$
\Sigma \mathbf{v}_k = \lambda_k \mathbf{v}_k,
$$
где $ \lambda_k $ — соответствующее собственное значение, характеризующее масштаб преобразования вдоль направления $ \mathbf{v}_k $. Собственные значения $ \lambda_k $ упорядочиваются по убыванию: $ \lambda_1 \geq \lambda_2 \geq \dots \geq \lambda_n $.


### 4. Первая главная компонента
Первая главная компонента определяется направлением, максимизирующим дисперсию. Рассмотрим задачу оптимизации:
$$
\max_{\mathbf{w}} \mathbf{w}^{\top} \Sigma \mathbf{w}, \quad \text{при} \quad \|\mathbf{w}\| = 1.
$$
Любой единичный вектор $ \mathbf{w} $ можно представить в виде линейной комбинации собственных векторов $ \mathbf{v}_k $:
$$
\mathbf{w} = \sum_{k} a_k \mathbf{v}_k, \quad \text{где} \quad \sum_{k} a_k^2 = 1.
$$
Подставляя это в выражение для дисперсии и учитывая ортогональность собственных векторов, получаем:
$$
\mathbf{w}^{\top} \Sigma \mathbf{w} = \left( \sum_{k} a_k \mathbf{v}_k \right)^{\top} \Sigma \left( \sum_{j} a_j \mathbf{v}_j \right) = \sum_{k} a_k^2 \lambda_k.
$$
Так как $ \sum_{k} a_k^2 = 1 $, значение $ \sum_{k} a_k^2 \lambda_k $ достигает максимума, равного $ \lambda_1 $ (наибольшему собственному значению), когда $ a_1 = 1 $, а все остальные $ a_k = 0 $ (для $ k \neq 1 $). Следовательно, $ \mathbf{w} = \mathbf{v}_1 $, где $ \mathbf{v}_1 $ — собственный вектор, соответствующий $ \lambda_1 $.


### 5. Последующие главные компоненты
Вторая главная компонента ищется в подпространстве, ортогональном к $ \mathbf{v}_1 $, путем решения аналогичной задачи максимизации дисперсии в этом подпространстве. Решением оказывается собственный вектор $ \mathbf{v}_2 $, соответствующий второму по величине собственному значению $ \lambda_2 $. Процесс продолжается рекурсивно: каждая последующая компонента $ \mathbf{v}_k $ ортогональна предыдущим и соответствует $ \lambda_k $, где $ k $ увеличивается по мере убывания $ \lambda_k $.

In [1]:
import math
import nampy as np


### Гаусс

In [2]:
def gauss_solver(A: np.array, b: np.array) -> np.array:
    rows, cols = A.shape
    if rows != b.shape[0]:
        raise ValueError(f"Размерности не совпадают: A({A.shape}), b({b.shape})")

    Ab = np.hstack([A, b])

    pivot_row = 0
    pivot_cols = []

    # Приведение к ступенчатому виду
    for j in range(cols):
        if pivot_row >= rows:
            break

        # Поиск максимального элемента в столбце j ниже pivot_row
        max_row = pivot_row
        for i in range(pivot_row + 1, rows):
            if abs(Ab[i, j]) > abs(Ab[max_row, j]):
                max_row = i
        Ab.swap(pivot_row, max_row)

        # Проверка на нулевой ведущий элемент
        pivot_val = Ab[pivot_row, j]
        if abs(pivot_val) < np.EPS:
            continue

        pivot_cols.append(j)

        # Нормализация ведущей строки
        Ab.div(pivot_row, pivot_val)

        # Обнуление элементов под ведущим элементом
        for i in range(rows):
            if i != pivot_row:
                factor = Ab[i, j]
                Ab.comb_rows(i, pivot_row, -factor)

        pivot_row += 1

    # Проверка на несовместность
    for i in range(pivot_row, rows):
        if abs(Ab[i, cols]) > np.EPS:
            raise ValueError("Система несовместна")

    rank = len(pivot_cols)

    # Единственное решение
    if rank == cols:
        solution = np.zeros((cols, 1))
        for i in range(rank - 1, -1, -1):
            pivot_col_idx = pivot_cols[i]
            val = Ab[i, cols]
            for k in range(pivot_col_idx + 1, cols):
                val -= Ab[i, k] * solution[k, 0]
            solution[pivot_col_idx, 0] = val
        return [np.array(solution.T)]

    # Бесконечно много решений
    elif rank < cols:
        # print(f"WARNING: для матрицы \n{A}\nвозвращается ФСР")
        free_vars_indices = [j for j in range(cols) if j not in pivot_cols]

        fsr_basis = []
        for _, free_idx in enumerate(free_vars_indices):
            basis_vector = np.zeros((cols, 1))

            # Устанавливаем текущую свободную переменную в 1, остальные свободные остаются 0
            basis_vector[free_idx, 0] = 1

            for j in range(rank - 1, -1, -1):
                pivot_col_idx = pivot_cols[j]
                val = 0
                for k in range(pivot_col_idx + 1, cols):
                    val -= Ab[j, k] * basis_vector[k, 0]
                basis_vector[pivot_col_idx, 0] = val

            fsr_basis.append(basis_vector.T)

        return fsr_basis

In [3]:
A = np.array([[2, 1, 1], [1, -1, 0], [3, -1, 2]])
b = np.array([2, -2, 2]).T
print("Case 1:")
sol = gauss_solver(A, b)
print(sol)

A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
b = np.array([[1], [2], [3]])
print("\nCase 2:")
sol = gauss_solver(A, b)
print(sol)


A = np.array([[2, 1, -1], [-3, -1, 2], [-2, 1, 2]])
b = np.array([[8], [-11], [-3]])
print("\nCase 3:")
sol = gauss_solver(A, b)
print(sol)


A = np.array([[1, 1], [1, 1]])
b = np.array([[1], [2]])
print("\nCase 4:")
try:
    sols = gauss_solver(A, b)
    print(sols)
except ValueError as e:
    print(e)

A = np.array([[1, 1, 1], [2, 2, 2]])
b = np.array([3, 6]).T
print("\nCase 5:")
sol = gauss_solver(A, b)
print(sol)

Case 1:
[[ -1.0000   1.0000   3.0000]]

Case 2:
[[  1.0000  -2.0000   1.0000]]

Case 3:
[[  2.0000   3.0000  -1.0000]]

Case 4:
Система несовместна

Case 5:
[[ -1.0000   1.0000   0.0000], [ -1.0000   0.0000   1.0000]]


In [4]:
def center_data(X: np.array) -> np.array:
    """
    Вход: матрица данных X (n*m)
    Выход: центрированная матрица X_centered (n*m)
    """
    mean_m = np.array([np.mean(X, axis=0)] * X.shape[0])
    return X - mean_m


q = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(center_data(q))

[ -3.0000  -3.0000  -3.0000]
[  0.0000   0.0000   0.0000]
[  3.0000   3.0000   3.0000]


In [5]:
def covariance_matrix(X: np.array) -> np.array:
    """
    Вход: центрированная матрица X (n*m)
    Выход: матрица ковариаций C (m*m)
    """
    return (1 / (X.shape[0] - 1)) * (X.T @ X)


X = covariance_matrix(center_data(q))
X

[  9.0000   9.0000   9.0000]
[  9.0000   9.0000   9.0000]
[  9.0000   9.0000   9.0000]

### Eigen


In [6]:
def gershgorin_interval(C: np.Matrix, eps: float = np.EPS) -> [float, float]:
    rows, cols = C.shape

    min_bound = np.INF
    max_bound = np.NINF

    for i in range(rows):
        diagonal_element = C[i, i]
        radius = 0
        for j in range(cols):
            if i != j:
                radius += abs(C[i, j])

        lower_i = diagonal_element - radius
        upper_i = diagonal_element + radius

        if lower_i < min_bound:
            min_bound = lower_i
        if upper_i > max_bound:
            max_bound = upper_i

    return min_bound - eps, max_bound + eps


def direct_det_func(C: np.Matrix, x: float) -> float:
    """
    Вычисляет значение характеристического полинома det(C - xI)
    """

    xI = np.eye(C.shape[0]) * x
    C_minus_xI = C - xI
    determinant = C_minus_xI.det
    return determinant


def find_all_roots(func, a, b, step: float = 0.01, tol: float = np.EPS) -> [float]:
    """
    Находит все корни функции func на интервале [a, b] методом деления отрезка пополам
    """
    roots = []
    x1 = a
    f1 = func(x1)

    if abs(f1) < tol:
        roots.append(x1)

    while x1 < b:
        x2 = min(x1 + step, b)
        f2 = func(x2)

        if abs(f2) < tol and abs(x2 - b) > tol:
            roots.append(x2)

        # Проверяем смену знака
        if f1 * f2 < 0:
            low = x1
            high = x2
            val_low = f1

            # Итерации бисекции
            while (high - low) / 2.0 > tol:
                mid = (low + high) / 2.0
                val_mid = func(mid)

                if abs(val_mid) < tol:
                    low = mid
                    high = mid
                    break

                if val_low * val_mid < 0:
                    high = mid
                else:
                    low = mid
                    val_low = val_mid

            found_root = (low + high) / 2.0

            is_duplicate = False
            for root in roots:
                if abs(root - found_root) < tol:
                    is_duplicate = True
                    break

            if not is_duplicate:
                roots.append(found_root)

        # Проверка на кратные корни
        elif abs(f1) < tol * 10 or abs(f2) < tol * 10:
            num_checks = 10
            sub_step = (x2 - x1) / num_checks
            for i in range(1, num_checks):
                x_check = x1 + i * sub_step
                f_check = func(x_check)
                if abs(f_check) < tol:
                    is_duplicate = False
                    for root in roots:
                        if abs(root - x_check) < tol:
                            is_duplicate = True
                            break
                    if not is_duplicate:
                        roots.append(x_check)

        x1 = x2
        f1 = f2

    # Проверяем, является ли конечная точка интервала корнем
    if abs(func(b)) < tol:
        is_duplicate = False
        for root in roots:
            if abs(root - b) < tol:
                is_duplicate = True
                break
        if not is_duplicate:
            roots.append(b)

    # Сортируем корни
    roots.sort()
    return roots

In [7]:
def find_eigenvalues(C: np.Matrix, tol: float = np.EPS) -> [float]:
    a, b = gershgorin_interval(C)

    def characteristic_polynomial(x: float) -> float:
        return direct_det_func(C, x)

    roots = find_all_roots(
        characteristic_polynomial, a, b, step=(b - a) / 10000, tol=tol
    )
    return roots


# C = np.array([[5, 6, 3], [-1, 0, 1], [1, 2, -1]])
# C = np.array([[4,1], [1,2]])
# C = np.array([[-1, -6], [2, 6]])
C = np.array([[5, -1, -1], [0, 4, -1], [0, -1, 4]])
eigenvalues = np.clip(np.array(find_eigenvalues(C)))
print(eigenvalues)

[       3        5]


In [8]:
def find_eigenvectors(C: np.array, eigenvalues: [float]) -> [np.array]:
    eigenvectors = []
    n = C.shape[0]
    I = np.eye(n)
    for lambada in eigenvalues:
        A = C - (lambada * I)

        x = gauss_solver(A, np.zeros((n, 1)))

        eigenvectors.append(x)
    return eigenvectors


eigenvectors = find_eigenvectors(C, eigenvalues)
for eig in eigenvectors:
    print(eig)

[[  1.0000   1.0000   1.0000]]
[[  1.0000   0.0000   0.0000], [  0.0000  -1.0000   1.0000]]


In [ ]:
def explained_variance_ratio(eigenvalues: [float], k: int) -> float:
    if not eigenvalues:
        raise ValueError("Eigenvalues array cannot be empty")
    if k <= 0:
        raise ValueError("k must be positive")
    if k > len(eigenvalues):
        raise ValueError("k cannot be greater than number of eigenvalues")
    if sum(eigenvalues) == 0:
        raise ValueError("Sum of eigenvalues cannot be zero")

    eigenvalues.sort(reverse=True)
    return sum(eigenvalues[:k]) / sum(eigenvalues)

### Hard

In [10]:
def pca(X: np.array, k: int) -> [np.array, float]:
    """
    Вход:
    X: матрица данных (n*m)
    k: число главных компонент
    Выход:
    X_proj: проекция данных (n*k)
    : доля объяснённой дисперсии
    """
    pass

In [11]:
from matplotlib.figure import Figure


def plot_pca_projection(X_proj: np.array) -> Figure:
    """
    Вход: проекция данных X_proj (n*2)
    Выход: объект Figure из Matplotlib
    """
    pass

In [12]:
def reconstruction_error(X_orig: np.array, X_recon: np.array) -> float:
    """
    Вход:
    X_orig: исходные данные (n*m)
    X_recon: восстановленные данные (n*m)
    Выход: среднеквадратическая ошибка MSE
    """
    pass

In [13]:
def auto_select_k(eigenvalues: [float], threshold: float = 0.95) -> int:
    """
    Вход:
    eigenvalues: список собственных значений
    threshold: порог объяснённой дисперсии
    Выход: оптимальное число главных компонент k
    """
    pass

In [14]:
def handle_missing_values(X: np.array) -> np.array:
    """
    Вход: матрица данных X (n*m) с возможными NaN
    Выход: матрица данных X_filled (n*m) без NaN
    """
    pass

In [15]:
def add_noise_and_compare(X: np.array, noise_level: float = 0.1):
    """
    Вход:
    X: матрица данных (n*m)
    noise_level: уровень шума (доля от стандартного отклонения)
    Выход: результаты PCA до и после добавления шума.
    В этом задании можете проявить творческие способности, поэтому выходные данные не
    →
    типизированы.
    """
    pass

In [16]:
# def apply_pca_to_dataset(dataset_name: str, k: int) -> [np.array, float]:
#     """
#     Вход:
#     dataset_name: название датасета
#     k: число главных компонент
#     Выход: кортеж (проекция данных, качество модели)
#     """
#     pass